In [ ]:
#import the necessary python libraries
import gpxpy
import gpxpy.gpx
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
from folium.vector_layers import Rectangle
from shapely.geometry import LineString
from shapely.ops import unary_union
import networkx as nx

#Read the gpx BMTC file and extract the coordinates in a particular segment
gpx_file = open('/Users/ashwin/Desktop/wards and gps/fefa3de2-c689-4017-a629-baf7e68f5d69_493.gpx', 'r')
gpx = gpxpy.parse(gpx_file)
qt2 = []

for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            qt2.append((point.latitude, point.longitude))

observations_array = np.array(qt2)
qt2 = observations_array[73:150]

#Plot the un-mapmatched route on the map
map_center2 = [gpx.tracks[0].segments[0].points[0].latitude, gpx.tracks[0].segments[0].points[0].longitude]
mymap2 = folium.Map(location=map_center2, zoom_start=12)

for track in gpx.tracks:
    for segment in track.segments:
        lat_lon_pairs2 = [(point.latitude, point.longitude) for point in segment.points[73:150]]  
        folium.PolyLine(lat_lon_pairs2, color="blue", weight=2.5, opacity=1).add_to(mymap2)

mymap2

#Make a geofence around the route on the map
all_points2 = []
for track in gpx.tracks:
    for segment in track.segments:
        all_points2.extend([(point.longitude, point.latitude) for point in segment.points[73:150]]) 

route_line2 = LineString(all_points2)
buffer_distance2 = 0.001  
route_buffer2 = route_line2.buffer(buffer_distance2)
map_center2 = route_line2.centroid.coords[0][::-1] 
mymap2 = folium.Map(location=map_center2, zoom_start=14)
lat_lon_pairs2 = [(point[1], point[0]) for point in route_line2.coords]
folium.PolyLine(lat_lon_pairs2, color="blue", weight=2.5, opacity=1).add_to(mymap2)
route_buffer_geojson2 = route_buffer2.__geo_interface__
folium.GeoJson(route_buffer_geojson2, name='geofence').add_to(mymap2)
mymap2

#Plot the same Geofence area on a OSMNX map
route_polygon2 = route_buffer2.convex_hull
G2 = ox.graph_from_polygon(route_polygon2, network_type='drive')
fig2, ax2 = ox.plot_graph(G2, bgcolor='w', edge_color='k', edge_linewidth=0.5, node_size=0, show=False, close=False)
route_line_buffer2 = route_line2.buffer(0.0001)  
ax2.plot(*route_line_buffer2.exterior.xy, color='blue', linewidth=1)
ax2.plot(*route_line_buffer2.exterior.xy[73:150], color='blue', linewidth=1)
plt.show()

#Extract the coordinates of the underlying graph of the map from the OSMNX map
route_polygon2 = route_buffer2.convex_hull
G2 = ox.graph_from_polygon(route_polygon2, network_type='drive')
states2 = np.array([(data2['y'], data2['x']) for node, data2 in G2.nodes(data=True)])
print("Number of nodes:", len(states2))
print("Node coordinates:")
for node in states2:
    print(node)

#Apply the Viterbi algortihm to get the Map Matched route to be taken
import numpy as np
from math import exp, sqrt, pi

pairwise_distances2 = []
for i in range(len(qt2) - 1):
    x1, y1 = qt2[i]
    x2, y2 = qt2[i + 1]
    distance = sqrt((x2 - x1)**2 + (y2 - y1)**2)
    pairwise_distances2.append(distance)
TRANS2 = np.zeros((len(states2), len(states2), len(qt2)-1))

beta = 0.95  

n2 = len(states2)
for k in range(len(qt2)-1):
    for i in range(n2):
        for j in range(n2):
            xtemp2 = np.linalg.norm(states2[i] - states2[j])
            dij2 = abs(xtemp2 - pairwise_distances2[k])
            TRANS2[i, j, k] = (1 / beta) * exp(-dij2 / beta)
        TRANS2[i, :, k] = TRANS2[i, :, k] / np.sum(TRANS2[i, :, k])

sigztemp2 = []
for i in range(len(qt2)):
    for j in range(len(states2)):
        sigztemp2.append(np.linalg.norm(qt2[i] - states2[j]))
sigz2 = 1.4 * np.median(sigztemp2)

EMIS2 = np.zeros((len(states2), len(qt2)))
for i in range(len(qt2)):
    for j in range(len(states2)):
        EMIS2[j, i] = (1 / (sqrt(2 * pi) * sigz2)) * exp(-0.5 * ((np.linalg.norm(qt2[i] - states2[j])) / sigz2) ** 2)

# seq = [1, 2, 3, 4]
seq2 = np.arange(1, len(qt2)+1)

PRIOR2 = np.array([(1 / len(states2))] * len(states2))
# PRIOR[358] = 0.7

T2 = len(seq2)  # Number of observations
n2 = EMIS2.shape[0]  # Number of states

score2 = np.zeros((n2, T2))
pred2 = np.zeros((n2, T2))

for i in range(n2):
    score2[i, 0] = PRIOR2[i] * EMIS2[i, seq2[0] - 1]

for t in range(1, T2):
    for j in range(n2):
        tscore2 = np.zeros(n2)
        for k in range(n2):
            tscore2[k] = score2[k, t - 1] * TRANS2[k, j, t - 1] * EMIS2[j, seq2[t] - 1]
        score2[j, t] = np.max(tscore2)
        id2 = np.where(tscore2 == np.max(tscore2))[0][0]
        pred2[j, t] = id2

IT2 = np.zeros(T2, dtype=int)
IT2[-1] = np.argmax(score2[:, -1])

for t in range(T2 - 1, 0, -1):
    IT2[t - 1] = int(pred2[IT2[t], t])

print(IT2)

#Calculate the shortest oath between the consecutive points to get the exact map match on a graph
def display_shortest_paths(paths, states):
    fig, ax = plt.subplots(figsize=(8, 8))  # Adjust figure size for better visualization
    for i, path in enumerate(paths):
        path_coordinates = [states[i] for i in path]
        x, y = zip(*path_coordinates)
        ax.plot(y, x, marker='o')  # Plot each path separately with a label
    ax.set_aspect('equal')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('Shortest Paths')
    ax.legend()  # Show legend to differentiate between paths
    plt.show()

G2 = nx.Graph()
for i in range(len(IT2) - 1):
    G2.add_edge(IT2[i], IT2[i + 1])
shortest_paths = [nx.shortest_path(G2, IT2[i], IT2[i + 1]) for i in range(len(IT2) - 1)]

# Display the shortest paths
display_shortest_paths(shortest_paths, states2)
